In [19]:
import os
from dotenv import load_dotenv

# Tải biến môi trường từ tệp .env
load_dotenv()

# Lấy khóa API từ biến môi trường
google_api_key = os.getenv("GOOGLE_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [20]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key

In [21]:
os.environ['OPENAI_API_KEY'] = google_api_key

In [1]:
import pandas as pd
import chromadb
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from chromadb.utils import embedding_functions
from langchain_chroma import Chroma

In [2]:


# Đọc dữ liệu từ file Excel
file_path = r'C:\Users\Admin\Documents\GPT_LLM\Final_Project\TPBankData.csv'  # Thay thế bằng đường dẫn thực tế đến file Excel của bạn
df = pd.read_csv(file_path)
# df.to_csv(r"C:\Users\Admin\Documents\GPT_LLM\Final_Project\TPBankData.csv", index=False)

# Kiểm tra dữ liệu đã đọc
print(df.head())




                                           Questions  \
0  Tài khoản của tôi có chắc chắn được đảm bảo và...   
1          Làm thế nào để đóng tài khoản tại TPBank?   
2  Làm thế nào để thay đổi tên chủ tài khoản tại ...   
3  Làm thế nào tôi biết được tài khoản của tôi đã...   
4  Liệu TPBank có chấp nhận mở tài khoản bằng ngo...   

                                             Answers  
0  Bằng việc áp dụng những công nghệ kỹ thuật mới...  
1  Để đóng tài khoản tại TPBank, Quý khách cần tr...  
2  Hiện nay, việc thay đổi tên chủ tài khoản (kèm...  
3  Thông thường, ngay sau khi hoàn tất thủ tục đă...  
4  Hiện tại ở TPBank, ngoài tài khoản bằng VND, Q...  


In [73]:
client = chromadb.HttpClient(host='localhost', port=8000)
client.delete_collection("QA_collection")

In [32]:
client = chromadb.PersistentClient(path=r"C:\Users\Admin\Documents\GPT_LLM\Final_Project\Database")

In [6]:
client = chromadb.HttpClient(host='localhost', port=8000)
embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

c:\Users\Admin\Documents\GPT_LLM\LLMVenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Admin\Documents\GPT_LLM\LLMVenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [74]:
# Tạo kết nối tới ChromaDB
client = chromadb.HttpClient(host='localhost', port=8000)  # Giả sử rằng bạn đã có thiết lập ChromaDB HttpClient
embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
# Tạo collection mới trong ChromaDB
collection = client.create_collection("QA_collection", embedding_function= embeddings)
# # Chuẩn bị dữ liệu để lưu trữ
# documents = df.iloc[:, 0].tolist()  # Cột câu hỏi (giả sử nằm ở cột thứ hai)
# answers = df.iloc[:, 1].tolist()    # Cột câu trả lời (giả sử nằm ở cột thứ nhất)
# ids = [f"doc{i}" for i in range(len(documents))]




for idx, row in df.iterrows():
    ques, ans = row["Questions"], row["Answers"]
    # Lưu trữ dữ liệu vào ChromaDB
    collection.add(
        documents=ques,
        metadatas= [{"Answers":ans}],
        ids=str(idx),
    )
    # print(ques, ":", ans)

In [7]:
db = client.get_collection(name="QA_collection", embedding_function=embeddings)

In [61]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db1 = Chroma(
    client=client,
    collection_name="QA_collection",
    embedding_function=embedding_function,
)

c:\Users\Admin\Documents\GPT_LLM\LLMVenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [62]:
query = "Làm thế nào để đóng tài khoản tại TPBank?"
docs = db1.similarity_search_with_score(query)

In [63]:
docs[0]

(Document(page_content='Làm thế nào để thay đổi tên chủ tài khoản tại TPBank? Hiện nay, việc thay đổi tên chủ tài khoản (kèm theo thay đổi số CMND/Hộ chiếu) không được chấp nhận thực hiện tại TPBank.'),
 0.4376767092197575)

In [16]:
# Ví dụ truy vấn
results = db.query(
    query_texts=["Trong thời gian bao lâu để xử lý tra soát, khiếu nại về dịch vụ Ngân hàng điện tử?"],
    n_results=1
)
# In kết quả truy vấn
print(results)

{'ids': [['27']], 'distances': [[0.05730263851186082]], 'embeddings': None, 'metadatas': [[{'Answers': 'Thời gian xử lý tra soát, khiếu nại phụ thuộc vào nội dung yêu cầu của khách hàng. TPBank cam kết sẽ hỗ trợ Quý khách trong thời gian sớm nhất có thể:\n• Với các giao dịch chuyển tiền nội bộ: Trong vòng 24h kể từ khi tiếp nhận yêu cầu\n• Với các giao dịch chuyển tiền thông thường tới ngân hàng khác, giao dịch nạp tiền điện thoại/thanh toán hóa đơn/mua hàng trực tuyến: Thời gian phụ thuộc vào thời gian xử lý của các đối tác, các trung gian thanh toán/Ngân hàng nhà nước và ngân hàng nhận tiền, tối đa 7 ngày làm việc kể từ khi TPBank tiếp nhận yêu cầu của Quý khách\n• Với các chuyển tiền nhanh tới ngân hàng khác: Thời gian xử lý phụ thuộc vào quy trình xử lý của đối tác thứ 3, có thể lên tới 15 ngày làm việc.'}]], 'documents': [['Thời gian xử lý tra soát, khiếu nại về dịch vụ Ngân hàng điện tử là bao lâu?']], 'uris': None, 'data': None}


In [17]:
answer = results['metadatas'][0][0].get('Answers', 'Không có câu trả lời')
print("Câu trả lời:", answer)

Câu trả lời: Thời gian xử lý tra soát, khiếu nại phụ thuộc vào nội dung yêu cầu của khách hàng. TPBank cam kết sẽ hỗ trợ Quý khách trong thời gian sớm nhất có thể:
• Với các giao dịch chuyển tiền nội bộ: Trong vòng 24h kể từ khi tiếp nhận yêu cầu
• Với các giao dịch chuyển tiền thông thường tới ngân hàng khác, giao dịch nạp tiền điện thoại/thanh toán hóa đơn/mua hàng trực tuyến: Thời gian phụ thuộc vào thời gian xử lý của các đối tác, các trung gian thanh toán/Ngân hàng nhà nước và ngân hàng nhận tiền, tối đa 7 ngày làm việc kể từ khi TPBank tiếp nhận yêu cầu của Quý khách
• Với các chuyển tiền nhanh tới ngân hàng khác: Thời gian xử lý phụ thuộc vào quy trình xử lý của đối tác thứ 3, có thể lên tới 15 ngày làm việc.


In [32]:
from langchain.prompts import PromptTemplate 
summarize_conversation_prompt = PromptTemplate(
    input_variables=["text","query"],
    template="Bạn là một chuyên gia trong các dịch vụ ngân hàng và cung cấp thông tin chính xác bằng đoạn văn dưới đây. \
    Câu trả lời của bạn nên chi tiết nhưng đơn giản, đảm bảo khách hàng hiểu tất cả các khía cạnh. \
    Giữ giọng điệu thân thiện và tránh các thuật ngữ kỹ thuật.\
    Câu hỏi: {query}\
    Nội dung: {text}"
)

In [59]:
user_input_prompt = PromptTemplate(input_variables=["user_input"],
                                 template="Người dùng trả lời: '{user_input}'. Họ có muốn cung cấp thông tin cá nhân để được hỗ trợ thêm không? Trả lời 'có' hoặc 'không'.")

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import LLMChain

In [67]:
LLM = ChatGoogleGenerativeAI(temperature=0.5, model="gemini-pro")

In [68]:
sum_chain = LLMChain(llm=LLM, prompt=summarize_conversation_prompt)
user_input_chain = LLMChain(llm = LLM, prompt=user_input_prompt)

In [71]:

query = input("\nBạn có muốn nhập thông tin muốn hỏi? (Trả lời tự nhiên): ").strip()
summary_result = sum_chain.run(query= query,text=answer)
summary_text = summary_result['text'] if isinstance(summary_result, dict) else summary_result
        


In [72]:
print(summary_text)

Thời gian chuyển tiền đến ngân hàng khác phụ thuộc vào loại giao dịch bạn thực hiện:

**Chuyển tiền nội bộ (trong cùng TPBank):**
* Trong vòng 24 giờ kể từ khi gửi yêu cầu

**Chuyển tiền thông thường đến ngân hàng khác:**
* Phụ thuộc vào thời gian xử lý của các ngân hàng khác, các đối tác thanh toán và Ngân hàng Nhà nước.
* Tối đa 7 ngày làm việc kể từ khi TPBank nhận được yêu cầu của bạn

**Chuyển tiền nhanh đến ngân hàng khác:**
* Phụ thuộc vào quy trình xử lý của bên thứ ba.
* Có thể mất đến 15 ngày làm việc


In [39]:
import csv

In [ ]:
def save_user_info(name, email, phone):
    with open('user_info.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name, email, phone])

In [62]:
# Hỏi xem người dùng có muốn nhập thông tin không
user_input = input("\nBạn có muốn nhập thông tin để được hỗ trợ thêm không? (Trả lời tự nhiên): ").strip()
    
# Sử dụng LLMChain để đánh giá ý định người dùng
intent = user_input_chain.run(user_input)

print(intent)

có


In [63]:
if intent == 'Có' or "có":
        # Yêu cầu người dùng nhập thông tin
    print("\nVui lòng nhập thông tin của bạn.")
    name = input("Tên: ")
    email = input("Email: ")
    phone = input("Số điện thoại: ")

    # Lưu thông tin người dùng
    # save_user_info(name, email, phone)
    print("Cảm ơn bạn! Thông tin của bạn đã được lưu lại.")
else:
    print("Cảm ơn bạn! Nếu cần hỗ trợ thêm, vui lòng liên hệ với chúng tôi.")


Vui lòng nhập thông tin của bạn.
Cảm ơn bạn! Thông tin của bạn đã được lưu lại.


### Xác định tin nhắn tiêu cực

In [64]:
negav_mess = PromptTemplate(input_variables=["message"],
    template="Phân loại tin nhắn sau là tích cực hay tiêu cực: '{message}'. Trả lời 'tích cực' hoặc 'tiêu cực'.")

In [65]:
negav_mess_chain = LLMChain(prompt=negav_mess, llm=LLM)

In [66]:
def classify_message_sentiment(message):
    sentiment = negav_mess_chain.run(message=message).strip().lower()
    return sentiment

# Ví dụ tin nhắn người dùng
messages = [
    "Tôi rất hài lòng với dịch vụ của bạn.",
    "Dịch vụ quá tệ và nhân viên không thân thiện.",
    "Sản phẩm chất lượng tốt, tôi sẽ mua lần sau.",
    "Giao hàng chậm trễ và sản phẩm bị hỏng."
]

# Đánh giá cảm xúc cho mỗi tin nhắn
for message in messages:
    sentiment = classify_message_sentiment(message)
    print(f"Tin nhắn: '{message}' - Cảm xúc: {sentiment}")

Tin nhắn: 'Tôi rất hài lòng với dịch vụ của bạn.' - Cảm xúc: tích cực
Tin nhắn: 'Dịch vụ quá tệ và nhân viên không thân thiện.' - Cảm xúc: tiêu cực
Tin nhắn: 'Sản phẩm chất lượng tốt, tôi sẽ mua lần sau.' - Cảm xúc: tích cực
Tin nhắn: 'Giao hàng chậm trễ và sản phẩm bị hỏng.' - Cảm xúc: tiêu cực
